In [67]:
# import library
import string
import pickle
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from util import JSONParser

In [68]:
# load data
path = "data/intents.json"

# buat objek JSONParser dan parse data intents.json
jp = JSONParser()
jp.parse(path)

# simpan dataframe dalam variabel df
df = jp.get_dataframe()

[DEBUG] Loaded JSON data: {'intents': [{'tag': 'salam', 'subintents': [{'tag': 'salam_pertanyaan', 'patterns': ['mw tanya weh', 'saya ada pertanyaan', 'Saya ingin bertanya', 'Mau tanya', 'Bantu saya'], 'responses': ['👋 *Halo! Selamat datang di layanan informasi pendaftaran UIB!*\n\nSaya siap membantu Anda dengan berbagai informasi terkait pendaftaran program S1, beasiswa, jadwal penting, dan banyak lagi. 😊\n\nBerikut beberapa hal yang bisa saya bantu:\n\n1️⃣ *Informasi Pendaftaran:* Saya dapat memberikan info tentang prosedur pendaftaran, persyaratan, dan dokumen yang dibutuhkan.\n2️⃣ *Beasiswa:* Cari tahu tentang beasiswa yang tersedia dan cara untuk mendaftar.\n3️⃣ *Jadwal dan Tenggat Waktu:* Dapatkan informasi tentang deadline pendaftaran dan acara penting lainnya.\n\n🔍 *Apa yang ingin Anda ketahui?* Silakan pilih topik atau tanyakan langsung, dan saya akan dengan senang hati membantu!', '👋 *Hai, terima kasih telah menghubungi kami!*\n\nSaya di sini untuk memberikan informasi yang A

In [69]:
# lihat 5 data pertama
df.head()

,text_input,intents
0,mw tanya weh,salam_pertanyaan
1,saya ada pertanyaan,salam_pertanyaan
2,Saya ingin bertanya,salam_pertanyaan
3,Mau tanya,salam_pertanyaan
4,Bantu saya,salam_pertanyaan


In [70]:
# hitung jumlah data per tag / inten
df.intents.value_counts()

intents
salam_umum                           16
bye_umum                             10
beasiswa_insan_mandiri                7
pembayaran_bank_ocbc                  7
beasiswa_cemerlang                    7
                                     ..
batas_waktu_pendaftaran_transfer      3
hubungi_humas_transfer                3
ujian_paket_c_lulusan_luar_negeri     3
pendaftaran_online_transfer           3
prosedur_sewa_stall_kantin            2
Name: count, Length: 64, dtype: int64

### Data preprocessing

- Mengubah semua alfabet menjadi huruf kecil
- Menghapus tanda baca

In [71]:
def preprocess(chat):
    # konversi ke lowercase
    chat = chat.lower()
    # menghapus tanda baca
    tandabaca = tuple(string.punctuation)
    chat = ''.join(ch for ch in chat if ch not in tandabaca)
    return chat

In [72]:
# implementasikan fungsi preprocess ke string
df['text_input_prep'] = df.text_input.apply(preprocess)

Apabila kita lihat hasilnya maka kita dapati hal berikut : 

In [73]:
df[['text_input', 'text_input_prep']].head(10)

,text_input,text_input_prep
0,mw tanya weh,mw tanya weh
1,saya ada pertanyaan,saya ada pertanyaan
2,Saya ingin bertanya,saya ingin bertanya
3,Mau tanya,mau tanya
4,Bantu saya,bantu saya
5,Hai,hai
6,Hi,hi
7,Halo,halo
8,Selamat Pagi,selamat pagi
9,Selamat Siang,selamat siang


### Vektorisasi

vektorisasi menggunakan metode bag of words yaitu mengumpulkan kata kata yang berasal dari korpus, dengan modul CountVectorizer dari scikit-learn

In [74]:
# inisiasi objek CountVectorizer
vect = CountVectorizer()

Langkah penting dalam metode *bag of words* adalah mengumpulkan *vocab* yang terdapat pada *corpus* yang kita miliki. Dalam scikit-learn kita dapat lakukan dengan cara :

In [75]:
# mengumpulkan vocab dari data teks yang sudah dilakukan praproses
vect.fit(df['text_input_prep'])

CountVectorizer()

In [76]:
# lihat list vocab
vect.get_feature_names_out()[:10] #batasi hanya 10 vocab teratas

array(['acara', 'ada', 'adakah', 'aja', 'akan', 'akhir', 'akuntansi',
       'anda', 'antar', 'antara'], dtype=object)

Selanjutnya konversi data teks menjadi matriks sesuai vocab yang sudah dibuat

In [77]:
# ubah data teks menjadi matriks
text_vect = vect.transform(df.text_input_prep)

text_vect

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 2219 stored elements and shape (315, 263)>

In [78]:
pd.DataFrame(text_vect.toarray(), columns=vect.get_feature_names_out())

,acara,ada,adakah,aja,akan,akhir,akuntansi,anda,antar,antara,...,ulangnya,untuk,usia,va,waktu,wawancara,weh,woe,ya,yang
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
311,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
312,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
313,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


### Modelling

Melakukan modelling dan data training menggunakan algoritma Multinomial Naive Bayes

In [79]:
# deklarasi objek MultinomialNB
nb = MultinomialNB()

# training data, dengan X : text_vect dan y : intents
nb.fit(text_vect, df.intents)

MultinomialNB()

model sudah dilatih, testing keluarkan hasil prediksi dari suatu input

In [80]:
# input string dari user
chat = input("Masukkan String : ")

# lakukan preproses
chat = preprocess(chat)

# ubah teks menjadi vektor
chat = vect.transform([chat])

# prediksi vektor teks kedalam model machine learning
res = nb.predict(chat)

# tampilkan hasil prediksi
print(f"Hasil prediksi : {res[0]}")

Hasil prediksi : salam_umum


prediksi dengan probabilitas jika hasil prediksi kurang dari threshold probabilitas = chatbot tidak mengerti

In [86]:
# input string dari user
chat = input("Masukkan String : ")

# lakukan preproses
chat = preprocess(chat)

# ubah teks menjadi vektor
chat = vect.transform([chat])

# prediksi vektor teks kedalam model machine learning
res = nb.predict_proba(chat)

# ambil nilai probabilitas tertinggi
max_prob = max(res[0])
max_idx = np.argmax(res[0])
print(f"Max Prob : {max_prob}\nMax Index: {max_idx}\nLabel: {nb.classes_[max_idx]}")

Max Prob : 0.05079365079365079
Max Index: 52
Label: salam_umum


### Efisiensi dengan Pipeline

Dari proses diatas kita bisa lihat apabila ada data teks maka kita perlu proses dalam dua langkah, yaitu vektorisasi dan pemodelan. Supaya proses menjadi lebih ringkas dan lebih mudah dalam proses deployment, kita akan buat pipeline

In [82]:
pipe = make_pipeline(CountVectorizer(),
                     MultinomialNB())

# Training
pipe.fit(df.text_input, df.intents)

Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('multinomialnb', MultinomialNB())])

Dapat kita lihat dari proses training diatas, seolah-olah kita langsung memasukkan data teks dan labelnya langsung kedalam "*black box*" sehingga proses prediksi akan lebih ringkas.

Untuk inference dengan pipeline dapat kita lakukan dengan cara berikut : 

In [102]:
# input string dari user
chat = input("Masukkan String : ")

# lakukan preproses
chat = preprocess(chat)

# prediksi teks kedalam pipeline
res = pipe.predict_proba([chat])

# ambil nilai probabilitas tertinggi
max_prob = max(res[0])
max_idx = np.argmax(res[0])
print(f"Max Prob : {max_prob}\nMax Index: {max_idx}\nLabel: {nb.classes_[max_idx]}")

Max Prob : 0.05079365079365079
Max Index: 52
Label: salam_umum


Dapat kita lihat bahwa teks baru setelah praproses bisa langsung masuk kedalam pipeline

### Simulasi Inference

Selanjutnya kita akan simulasikan chatbot mulai dari mendapatkan input sampai ke respon

Dalam kasus ini apabila intent yang terdeteksi adalah `bye` maka program berhenti

In [98]:
print("Anda Terhubung dengan chatbot Kami")
while True:
    # input user
    chat = input("Anda : ")
    # praproses input
    chat = preprocess(chat)
    
    # prediksi intent dengan pipeline
    res = pipe.predict_proba([chat])
    
    # ambil probabilitas tertinggi dan indeksnya
    max_prob = max(res[0])
    max_idx = np.argmax(res[0])
    intent = pipe.classes_[max_idx]
    
    # kondisi jika probabilitas kurang dari threshold
    if max_prob < 0.05:
        print("Bot : Maaf, saya tidak mengerti, jika anda butuh bantuan harap menghubungi humas kami.")
    else:
        # gunakan tag intent atau subintent yang terdeteksi untuk mendapatkan respons
        response = jp.get_response(intent)
        print(f"Bot : {response}")
    
    if intent == 'bye' or intent == 'bye_umum':
        break


Anda Terhubung dengan chatbot Kami
Bot : 🌟 *Terima kasih telah menghubungi kami! Sampai jumpa!* 

Kami berharap Anda puas dengan bantuan yang diberikan. Jika ada pertanyaan lainnya atau jika Anda membutuhkan dukungan lebih lanjut, kami selalu siap membantu. Anda dapat menghubungi kami melalui:

📱 *LINE Pusat Informasi UIB:* [Link LINE]
📞 *WhatsApp Pusat Informasi UIB:* [Nomor WhatsApp]

Jangan ragu untuk kembali jika ada yang perlu Anda tanyakan! Semoga harimu menyenangkan! 😊


Setelah kita berhasil simulasikan bot nya dalam notebook ini, kita simpan modelnya agar dapat dideploy dengan mudah

In [85]:
with open("chatbot_pipeline.pkl", "wb") as model_file:
    pickle.dump(pipe, model_file)